In [1]:
%matplotlib widget
import matplotlib
import sys
sys.path.append('/usr/wrk/people9/sima9999/git/pyOMA')
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt 
import ipywidgets
import ipympl.backend_nbagg # the ''%matplotlib widget' backend
#from matplotlib.backends.backend_nbagg import FigureCanvasNbAgg # the ''%matplotlib notebook' backend

from core.PreprocessingTools import PreprocessData, GeometryProcessor
from core.ModalBase import ModalBase
from core.PLSCF import PLSCF
from core.PRCE import PRCE
from core.SSICovRef import BRSSICovRef, PogerSSICovRef
from core.SSIData import SSIData, SSIDataMC
from core.VarSSIRef import VarSSIRef
from core.StabilDiagram import StabilCalc, StabilCluster, StabilPlot, DataCursor
from core.PostProcessingTools import MergePoSER
from core.PlotMSH import ModeShapePlot

In [2]:
PreprocessData.load_measurement_file = np.load

working_dir = Path('/usr/wrk/people9/sima9999/git/pyOMA/tests/files/')

geometry_data = GeometryProcessor.load_geometry(
    nodes_file=working_dir / 'grid.txt',
    lines_file=working_dir / 'lines.txt')

result_folder = Path('/usr/wrk/people9/sima9999/git/pyOMA/tests/files/measurement_1/')
meas_name = os.path.basename(result_folder)

setup_info=result_folder / 'setup_info.txt'
meas_file=result_folder / (meas_name + '.npy')
conf_file=working_dir / 'varssi_config.txt'
method=BRSSICovRef
geometry_data=geometry_data
chan_dofs_file=result_folder / "channel_dofs.txt"
skip_existing=True
save_results=False
interactive=True

In [3]:
if not os.path.exists(result_folder / 'prep_data.npz') or not skip_existing:
    prep_data = PreprocessData.init_from_config(
        conf_file=setup_info,
        meas_file=meas_file,
        chan_dofs_file=chan_dofs_file)

    if save_results:
        prep_data.save_state(result_folder / 'prep_data.npz')
else:
    prep_data = PreprocessData.load_state(result_folder / 'prep_data.npz')

if not os.path.exists(
        result_folder /
        'modal_data.npz') or not skip_existing:

    modal_data = method.init_from_config(conf_file, prep_data)

    if save_results:
        modal_data.save_state(result_folder / 'modal_data.npz')
else:
    modal_data = method.load_state(
        result_folder / 'modal_data.npz', prep_data)

if os.path.exists(result_folder / 'stabil_data.npz') and skip_existing:
    stabil_calc = StabilCalc.load_state(
        result_folder / 'stabil_data.npz', modal_data, prep_data)
else:
    stabil_calc = StabilCalc(modal_data, prep_data)

    

INFO:core.PreprocessingTools:Now loading previous results from  /usr/wrk/people9/sima9999/git/pyOMA/tests/files/measurement_1/prep_data.npz
Now loading previous results from  /usr/wrk/people9/sima9999/git/pyOMA/tests/files/measurement_1/modal_data.npz
Covariance Matrices Built
State Matrices Computed
Modal Parameters Computed
Now loading previous results from  /usr/wrk/people9/sima9999/git/pyOMA/tests/files/measurement_1/stabil_data.npz


In [4]:
def update_value_view(i):
    n, f, stdf, d, stdd, mpc, mp, mpd, dmp, dmpd, mtn, MC, ex_1, ex_2 = stabil_calc.get_modal_values(i)

    if stabil_calc.capabilities['std']:
        import scipy.stats
        num_blocks = stabil_calc.modal_data.num_blocks
        stdf = scipy.stats.t.ppf(
            0.975, num_blocks) * stdf / np.sqrt(num_blocks)
        stdd = scipy.stats.t.ppf(
            0.975, num_blocks) * stdd / np.sqrt(num_blocks)

    current_mode = i
    s = ''
    for text, val in [('Frequency=%1.3fHz, \n' % (f), f),
                      ('CI Frequency ± %1.3e, \n' % (stdf), stdf),
                      ('Order=%1.0f, \n' % (n), n),
                      ('Damping=%1.3f%%,  \n' % (d), d),
                      ('CI Damping ± %1.3e,  \n' % (stdd), stdd),
                      ('MPC=%1.5f, \n' % (mpc), mpc),
                      ('MP=%1.3f\u00b0, \n' % (mp), mp),
                      ('MPD=%1.5f\u00b0, \n' % (mpd), mpd),
                      ('dMP=%1.3f\u00b0, \n' % (dmp), dmp),
                      #('dMPD=%1.5f\u00b0, \n' % (dmpd),     dmpd),
                      ('MTN=%1.5f, \n' % (mtn), mtn),
                      ('MC=%1.5f, \n' % (MC), MC),
                      ('Ext=%1.5f\u00b0, \n' % (ex_1), ex_1),
                      ('Ext=%1.3f\u00b0, \n' % (ex_2), ex_2)
                      ]:
        if val is not np.nan:
            s += text
    current_value_view.value = s
    
    return

In [8]:
if interactive:
    stabil_plot = StabilPlot(stabil_calc)
    stabil_plot.update_stabilization() # initialize the plot
    
    print('Which backend are we using? ', plt.get_backend())

    df_max = stabil_calc.df_max * 100
    dd_max = stabil_calc.dd_max * 100
    dmac_max = stabil_calc.dmac_max * 100
    d_range = stabil_calc.d_range
    mpc_min = stabil_calc.mpc_min
    mpd_max = stabil_calc.mpd_max


    freq_range = ipywidgets.FloatRangeSlider(
        orientation='horizontal',
        description='Frequency Range:',
        value=stabil_plot.ax.get_xlim(),
        min=0,
        max=prep_data.sampling_rate/2,
        step=prep_data.sampling_rate/2/100,
        readout=False
    )
    df_edit = ipywidgets.FloatText(
        value=df_max,
        description='Frequency [%]:',
        disabled=False
    )
    
    current_value_view = ipywidgets.Textarea(
        value='',
        placeholder='',
        description='String:',
        disabled=False
        )

    freq_range.layout.margin = '0px 30% 0px 30%'
    freq_range.layout.width = '40%'

    fig = stabil_plot.fig
    canvas = ipympl.backend_nbagg.Canvas(fig)
    manager = ipympl.backend_nbagg.FigureManager(canvas, 0)
    canvas.header_visible = False
    canvas.toolbar_position = 'right'
    
    cursor = stabil_plot.init_cursor(False)
    cursor.add_callback('show_current_info', update_value_view)
    
    
    def update_xlim(change):
        xlim = change.new
        stabil_plot.ax.set_xlim(xlim)
        redraw()

    def update_df(change):
        df_max = change.new/100
        stabil_plot.update_stabilization(df_max=df_max)
        redraw()

    def redraw():
        fig.canvas.draw()
        fig.canvas.flush_events()

    freq_range.observe(update_xlim, names='value')
    df_edit.observe(update_df, names='value')

    grid = ipywidgets.GridspecLayout(4, 8)
    grid[:3, 1:] = fig.canvas
    grid[0, 0] = df_edit
    grid[1, 0] = current_value_view
    grid[3, 1:] = freq_range

    display(grid)

Which backend are we using?  module://ipympl.backend_nbagg


GridspecLayout(children=(Canvas(header_visible=False, layout=Layout(grid_area='widget001', min_height='600px')…

In [6]:
if save_results:
    stabil_calc.save_state(result_folder / 'stabil_data.npz')